In [3]:
df_raw = spark.read.option("header","true").option("inferSchema","true").csv("Files/raw/Sample - Superstore.csv")

df_raw.show(5)
df_raw.printSchema()

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 5, Finished, Available, Finished)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [4]:
from pyspark.sql.functions import col, to_date,datediff, current_date

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 6, Finished, Available, Finished)

In [10]:
df_raw.show(5)

StatementMeta(, 94adfed0-8a41-4700-b6e4-842a33e02ddd, 12, Finished, Available, Finished)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [1]:
spark.conf.set(
    "spark.sql.legacy.timeParserPolicy",
    "LEGACY")

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 3, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import col, to_date, datediff

df = (
    df_raw
    .withColumn("Order Date", to_date(col("Order Date"), "MM/dd/yyyy"))
    .withColumn("Ship Date", to_date(col("Ship Date"), "MM/dd/yyyy"))
    .dropDuplicates()
)
df = df.withColumn(
    "Shipping_Days",
    datediff(col("Ship Date"), col("Order Date"))
)
display(
    df.select(
        "Order ID","Customer ID","Sales","Quantity","Profit","Shipping_Days").limit(5))

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 37bacc1e-aae0-4d9f-9d7e-1d95e256a689)

In [ ]:
fact_sales = df.select(col("Order ID").alias("order_id"),col("Order Date").alias("order_date"),col("Ship Date").alias("ship_date"),
    col("Customer ID").alias("customer_id"),col("Product ID").alias("product_id"),col("Category"),col("Sub-Category").alias("sub_category"),
    col("Region"),col("Sales").cast("double"),col("Quantity").cast("int"),col("Profit").cast("double"),col("Shipping_Days"))

In [ ]:
fact_sales.write.format("delta").mode("overwrite").saveAsTable("fact_sales")

In [11]:
display(spark.sql("SELECT * FROM fact_sales LIMIT 5"))

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 58757a08-f394-4d90-a423-a87c0b03b39a)

In [14]:
dim_customer = df.select(col("Customer ID").alias("customer_id"),col("Customer Name").alias("customer_name"),
    col("Segment"),col("Region")).dropDuplicates()

dim_customer.write.format("delta").mode("overwrite").saveAsTable("dim_customer")

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 16, Finished, Available, Finished)

In [15]:
display(spark.sql("SELECT * FROM dim_customer LIMIT 5"))

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4fe978ae-7f0c-49eb-960a-a5c1a46f5582)

In [17]:
dim_product = df.select(col("Product ID").alias("product_id"),col("Product Name").alias("product_name"),
    col("Category"),col("Sub-Category").alias("sub_category")).dropDuplicates()

dim_product.write.format("delta").mode("overwrite").saveAsTable("dim_product")

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 19, Finished, Available, Finished)

In [18]:
display(spark.sql("SELECT * FROM dim_product LIMIT 5"))

StatementMeta(, 903d646e-c8e0-4736-9d2d-e70d90ffaee5, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 299d98ab-22e6-4306-834e-bc544accadfe)

In [1]:
from pyspark.sql.functions import col, max as spark_max

dim_product_df = spark.table("dbo.dim_product")

dim_product_clean_df = (dim_product_df.groupBy("product_id")
    .agg(spark_max("product_name").alias("product_name"),
        spark_max("category").alias("category"),
        spark_max("sub_category").alias("sub_category")
    ))

StatementMeta(, 0d4ece51-c039-4e5f-b26e-7354911f8f1f, 3, Finished, Available, Finished)

In [2]:
dim_product_clean_df.write.format("delta").mode("overwrite").saveAsTable("dbo.dim_product_clean")

print("dim_product_clean table created successfully")


StatementMeta(, 0d4ece51-c039-4e5f-b26e-7354911f8f1f, 4, Finished, Available, Finished)

dim_product_clean table created successfully


In [3]:
spark.sql("SELECT COUNT(*) FROM dbo.dim_product_clean").show()

StatementMeta(, 0d4ece51-c039-4e5f-b26e-7354911f8f1f, 5, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
|    1862|
+--------+



In [1]:

from pyspark.sql.functions import col, max as spark_max

dim_customer_df = spark.table("dbo.dim_customer")

dim_customer_clean_df = (dim_customer_df.groupBy("customer_id")
    .agg(spark_max("customer_name").alias("customer_name"),
        spark_max("Region").alias("Region"),
        spark_max("Segment").alias("Segment")
    ))

StatementMeta(, c034db89-da84-427f-be4d-89d13fa6372e, 3, Finished, Available, Finished)

In [2]:
dim_customer_clean_df.write.format("delta").mode("overwrite").saveAsTable("dbo.dim_customer_clean")
print("dim_customer_clean table created successfully")

StatementMeta(, c034db89-da84-427f-be4d-89d13fa6372e, 4, Finished, Available, Finished)

dim_customer_clean table created successfully


In [3]:
spark.sql("SELECT COUNT(*) FROM dbo.dim_customer_clean").show()

StatementMeta(, c034db89-da84-427f-be4d-89d13fa6372e, 5, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
|     793|
+--------+

